In [7]:
from sagemaker.pytorch import PyTorchModel
import sagemaker
from datetime import datetime
import time
import boto3
import os

s3_client = boto3.client('s3')
smsess = sagemaker.Session()
region = smsess.boto_region_name
role = sagemaker.get_execution_role()


## Pull changes, create Tar and upload to s3

In [ ]:
!git pull

In [ ]:
!tar -czvf model.tar.gz code/ model.pt
print("Compress finished")

In [ ]:
bucket_name = 'beats-model-artifacts'
file_to_delete = 'model.tar.gz'
file_to_upload = 'model.tar.gz'
upload_key = 'model.tar.gz'

In [ ]:
s3_client.delete_object(Bucket=bucket_name, Key=file_to_delete)

In [ ]:
s3_client.upload_file(file_to_upload, bucket_name, upload_key)

## Model Loading

In [ ]:
model_name = "model.pt"
model_data = "s3://beats-model-artifacts/model.tar.gz"

In [55]:
model = PyTorchModel(entry_point="inference.py",
model_data=model_data,
framework_version="1.12",
py_version="py38",
env={"MODEL_NAME": model_name},
role=role,
sagemaker_session=smsess)

In [56]:
INSTANCE_TYPE = "ml.m5.4xlarge"
ENDPOINT_NAME = "beats-pytorch-" + str(datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S-%f"))

In [57]:
predictor = model.deploy(initial_instance_count=1,
instance_type=INSTANCE_TYPE,
deserializer=sagemaker.deserializers.JSONDeserializer(),
endpoint_name=ENDPOINT_NAME)

----!

In [58]:
start_time = time.time()
predicted_value = predictor.predict("ESC-50.wav")
duration = time.time() - start_time

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/beats-pytorch-2023-07-25-18-45-45-494234 in account 410677554255 for more information.

In [ ]:
predicted_value

In [ ]:
duration

In [59]:
predictor.delete_endpoint(delete_endpoint_config=True)